# 🎓 Radyoloji LLM Karşılaştırma - Öğrenci Şablonu

Bu notebook, Radyoloji LLM'lerini karşılaştırma projesi için bir başlangıç noktasıdır.
Gerekli `src` modülleri proje dizininde hazırdır. Sizin göreviniz bu modülleri kullanarak bir deney kurgulamaktır.

**Hedefler:**
1. Proje ortamını kurmak (Google Drive mount, bağımlılıklar).
2. `src.utils.model_registry` kullanarak modelleri listelemek.
3. Seçilen açık kaynak bir modeli (örn. Qwen2-VL) yüklemek.
4. Örnek bir göğüs röntgeni üzerinde rapor üretmek.
5. Farklı prompt'ların (baseline, detailed) çıktılarını karşılaştırmak.

---

## 1️⃣ Kurulum ve Ortam Hazırlığı

**Görev:**
* Google Drive'ı bağlayın.
* Proje dizinine gidin (`src` klasörünün olduğu yer).
* Gerekli kütüphaneleri (`requirements.txt` veya manuel) yükleyin.
* GPU'nun aktif olduğunu doğrulayın.

In [ ]:
# TODO: Google Drive mount işlemi

# TODO: Proje dizinine geçiş ve path ayarları

# TODO: GPU kontrolü

## 2️⃣ Model Registry Keşfi

**Görev:**
* `src.utils.model_registry` modülünü inceleyin.
* Mevcut modelleri listeleyen fonksiyonu çağırın.

In [ ]:
# TODO: Modelleri listeleyin

## 3️⃣ Prompt Yönetimi

**Görev:**
* `src.utils.prompt_manager.PromptManager` sınıfını kullanarak mevcut promptları listeleyin.

In [ ]:
# TODO: PromptManager'ı başlatın ve promptları listeleyin

## 4️⃣ Model Yükleme ve Rapor Üretimi

**Görev:**
* `transformers` kütüphanesini kullanarak `Qwen/Qwen2-VL-2B-Instruct` modelini yükleyin.
* `src/img` altındaki test görüntüsünü yükleyin.
* Modelden bir rapor üretin.

In [ ]:
# TODO: Modeli yükleyin (AutoProcessor ve AutoModelForCausalLM/ConditionalGeneration)

# TODO: Görüntüyü yükleyin

# TODO: Raporu oluşturun ve yazdırın

## 5️⃣ Deney: Prompt Etkisi

**Görev:**
* Aynı görüntü için "baseline", "detailed" ve "turkish" promptlarını kullanarak 3 farklı rapor üretin.
* Sonuçları kıyaslayın.

In [ ]:
# TODO: Döngü içinde promptları deneyin ve sonuçları basın

## 6️⃣ (Opsiyonel) Değerlendirme ve Kayıt

**Görev:**
* Sonuçları bir JSON dosyasına kaydedin.

In [ ]:
# TODO: Sonuçları kaydet